Import

In [1]:
import os
from contextlib import redirect_stdout

import sys
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt
import re
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu
from collections import OrderedDict

## Paths 

In [2]:
def find_calsim_base_path(start_path, target_folder="CalSim3_Model_Runs"):
    # Go up twice to reach DSP, then look for sibling folder
    current_path = os.path.abspath(start_path)
    dsp_root = os.path.dirname(os.path.dirname(current_path))  # notebooks → coeqwal → DSP
    candidate = os.path.join(dsp_root, target_folder)
    if os.path.isdir(candidate):
        return candidate
    raise FileNotFoundError(f"{target_folder} not found alongside {dsp_root}")

base_dir = os.path.abspath(".")
calsim_base_path = find_calsim_base_path(base_dir)

salinity_tiers_output_dir = os.path.join(
    calsim_base_path,
    "Scenarios",
    "Performance_Metrics",
    "Tiered_Outcome_Measures",
    "Salinity",
    "Tiers"
)

salinity_data_output_dir = os.path.join(
    calsim_base_path,
    "Scenarios",
    "Performance_Metrics",
    "Metrics",
    "Salinity"
)
os.makedirs(salinity_tiers_output_dir, exist_ok=True)
os.makedirs(salinity_data_output_dir, exist_ok=True)


## File names

In [3]:
april_x2_ann_avg_path = os.path.join(salinity_data_output_dir, "AprilX2_AnnualAverage.csv")
april_x2_ann_cv_path = os.path.join(salinity_data_output_dir, "AprilX2_AnnualCV.csv")
september_x2_ann_avg_path = os.path.join(salinity_data_output_dir, "SeptemberX2_AnnualAverage.csv")
september_x2_ann_cv_path = os.path.join(salinity_data_output_dir, "SeptemberX2_AnnualCV.csv")
salinity_compliance_points_indelta_path = os.path.join(salinity_data_output_dir, "InDeltaSalinity.csv")
salinity_compliance_points_export_path = os.path.join(salinity_data_output_dir, "ExportSalinity.csv")
indelta_salinity_output_path = os.path.join(salinity_tiers_output_dir, "InDeltaTierAssignment.csv")
export_salinity_output_path = os.path.join(salinity_tiers_output_dir, "ExportTierAssignment.csv")


### Initialize

In [4]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'

ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

### Read scenario indices

In [5]:
indexhdr, index_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, IndexMin, IndexMax, hdr=True)
index_names = []
for i in range(len(index_name)):
    index_names.append(index_name[i][0])
index_names

['s0001',
 's0002',
 's0003',
 's0004',
 's0005',
 's0006',
 's0007',
 's0008',
 's0009',
 's0010',
 's0011',
 's0012',
 's0013',
 's0014',
 's0015',
 's0016',
 's0018',
 's0019',
 's0020',
 's0021',
 's0022',
 's0023',
 's0024',
 's0025',
 's0027',
 's0029',
 's0030',
 's0039',
 's0042',
 's0046',
 's0047',
 's0051',
 's0056',
 's0062',
 's0063']

### Read dataset

In [6]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)
df = add_water_year_column(df)
df

C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\metrics.py:174: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


A                        CALSIM                                              \
B          AWOANN_64_XADV_s0001 AWOANN_72_XA1DV_s0001 AWOANN_72_XA2DV_s0001   
C          ANNUAL-APPLIED-WATER  ANNUAL-APPLIED-WATER  ANNUAL-APPLIED-WATER   
D                          1MON                  1MON                  1MON   
E                        L2020A                L2020A                L2020A   
F                      PER-AVER              PER-AVER              PER-AVER   
Units                       TAF                   TAF                   TAF   
1921-10-31             59.04970             136.83063             331.60556   
1921-11-30             59.04970             136.83063             331.60556   
1921-12-31             59.04970             136.83063             331.60556   
1922-01-31             59.04970             136.83063             331.60556   
1922-02-28             59.04970             136.83063             331.60556   
..                          ...                   ...                   ...   
2021-05-31             60.19477             140.76320             343.78314   
2021-06-30             60.19477             140.76320             343.78314   
2021-07-31             60.19477             140.76320             343.78314   
2021-08-31             60.19477             140.76320             343.78314   
2021-09-30             60.19477             140.76320             343.78314   

A                                                                             \
B          AWOANN_72_XA3DV_s0001 AWOANN_73_XADV_s0001 BANKSEC_MAX14DAY_s0001   
C           ANNUAL-APPLIED-WATER ANNUAL-APPLIED-WATER        SALINITY-APPROX   
D                           1MON                 1MON                   1MON   
E                         L2020A               L2020A                 L2020A   
F                       PER-AVER             PER-AVER               PER-AVER   
Units                        TAF                  TAF               UMHOS/CM   
1921-10-31             128.51282            65.573940              284.73438   
1921-11-30             128.51282            65.573940              281.05875   
1921-12-31             128.51282            65.573940              376.29092   
1922-01-31             128.51282            65.573940              623.95900   
1922-02-28             128.51282            65.573940              416.49713   
..                           ...                  ...                    ...   
2021-05-31             130.04979            69.374054              393.38092   
2021-06-30             130.04979            69.374054              383.89062   
2021-07-31             130.04979            69.374054              446.86966   
2021-08-31             130.04979            69.374054              464.96625   
2021-09-30             130.04979            69.374054              438.54697   

A                                                             \
B          COREQSACDV_s0001 CO_EC_MONTH_s0001 C_AMR004_s0001   
C                      FLOW          SALINITY        CHANNEL   
D                      1MON              1MON           1MON   
E                    L2020A            L2020A         L2020A   
F                  PER-AVER          PER-AVER       PER-AVER   
Units                   CFS          UMHOS/CM            CFS   
1921-10-31           0.0000         2173.8489     2164.72950   
1921-11-30           0.0000         2621.5984     2206.79600   
1921-12-31        4017.6848         5590.7207     2024.89980   
1922-01-31           0.0000        10744.2570     1704.54830   
1922-02-28           0.0000         3148.9802     5005.48340   
..                      ...               ...            ...   
2021-05-31           0.0000         4102.8306      645.44150   
2021-06-30           0.0000         6367.1660     1839.48110   
2021-07-31           0.0000         8060.9453     1755.98750   
2021-08-31           0.0000         8370.9795     1914.82500   
2021-09-30           0.0000         8845.4890   

### Define variables

In [7]:
in_delta_vars = ["EM_EC_MONTH", "JP_EC_MONTH"]
export_vars = ["TRACYEC_MAX14DAY", "BANKSEC_MAX14DAY"]
indelta_thresholds={"Top": 2500, "Mid": 1600, "Low": 900}
export_thresholds={"Top": 2500, "Mid": 1600, "Low": 900}
indelta_station_list=["EM", "JP"]
export_station_list=["BANKSEC", "TRACYEC"]
indelta_rules = OrderedDict([
    (1, {"LT_A": 0.75, "LT_B": None, "GT_C": 0.05}),
    (2, {"LT_A": 0.65, "LT_B": 0.75, "GT_C": 0.12}),
    (3, {"LT_A": 0.55, "LT_B": 0.65, "GT_C": 0.20}),
])
x2 = 'X2_PRV_KM'
compliance_points_indelta = ["EM_EC_MONTH", "JP_EC_MONTH", "RS_EC_MONTH", "CO_EC_MONTH"]
compliance_points_export = ["BANKSEC", "TRACYEC"]

### Subset data set

In [8]:
in_delta_df = create_subset_list(df, in_delta_vars)
in_delta_df

A                     CALSIM                                      \
B          EM_EC_MONTH_s0001 JP_EC_MONTH_s0001 EM_EC_MONTH_s0002   
C                   SALINITY          SALINITY          SALINITY   
D                       1MON              1MON              1MON   
E                     L2020A            L2020A            L2020A   
F                   PER-AVER          PER-AVER          PER-AVER   
Units               UMHOS/CM          UMHOS/CM          UMHOS/CM   
1921-10-31         420.35240         308.33957         420.35240   
1921-11-30         561.92170         298.96088         560.35170   
1921-12-31        1132.24480        1211.59310        1200.42440   
1922-01-31         713.35815        1374.92550         724.70953   
1922-02-28         359.12146         654.43500         345.48660   
..                       ...               ...               ...   
2021-05-31         849.52420         517.29944         838.77576   
2021-06-30        1616.85250         842.68225        1669.95700   
2021-07-31        2524.26300        1285.16800        2613.50560   
2021-08-31        2431.64900        1367.02320        2499.81080   
2021-09-30        2797.95000        1376.98610        2753.90580   

A                                                                 \
B          JP_EC_MONTH_s0002 EM_EC_MONTH_s0003 JP_EC_MONTH_s0003   
C                   SALINITY          SALINITY          SALINITY   
D                       1MON              1MON              1MON   
E                     L2020A            L2020A            L2020A   
F                   PER-AVER          PER-AVER          PER-AVER   
Units               UMHOS/CM          UMHOS/CM          UMHOS/CM   
1921-10-31         308.33957         378.06560         269.90536   
1921-11-30         299.63480         478.07477         248.80586   
1921-12-31        1283.97570        1018.15460         836.14660   
1922-01-31        1500.18680         379.06232         840.27026   
1922-02-28         656.80850         214.80246         287.58038   
..                       ...               ...               ...   
2021-05-31         506.96332         875.91693         493.00034   
2021-06-30         872.56220        2112.61400         846.22380   
2021-07-31        1316.08480        2823.19630        1496.07540   
2021-08-31        1346.60570        2629.35770        1401.00840   
2021-09-30        1318.96280        2826.85130        1325.63900   

A                                                                 \
B          EM_EC_MONTH_s0004 JP_EC_MONTH_s0004 EM_EC_MONTH_s0005   
C                   SALINITY          SALINITY          SALINITY   
D                       1MON              1MON              1MON   
E                     L2020A            L2020A            L2020A   
F                   PER-AVER          PER-AVER          PER-AVER   
Units               UMHOS/CM          UMHOS/CM          UMHOS/CM   
1921-10-31         403.16763         253.26294         403.16763   
1921-11-30         436.82724         245.62279         433.17044   
1921-12-31         998.01874         863.27500        1009.44750   
1922-01-31         338.66785         742.52374         380.75513   
1922-02-28         243.60349         300.92972         268.10992   
..                       ...               ...               ...   
2021-05-31        1032.20580         498.76810        1138.69620   
2021-06-30        2514.34940         973.80164        2580.97020   
2021-07-31        2750.93070        1495.42990        2718.73780   
2021-08-31        2567.57520        1231.75630        2655.68460   
2021-09-30        2621.02930        1204.72670        2633.80660   

A                             ...                                      \
B          JP_EC_MONTH_s0005  ... EM_EC_MONTH_s0047 JP_EC_MONTH_s0047   
C                   SALINITY  ...          SALINITY          SALINITY   
D                       1MON  ...              1MON              1MON   
E                     L2020A  ...

In [9]:
export_df = create_subset_list(df, export_vars)
export_df

A                          CALSIM                         \
B          BANKSEC_MAX14DAY_s0001 TRACYEC_MAX14DAY_s0001   
C                 SALINITY-APPROX        SALINITY-APPROX   
D                            1MON                   1MON   
E                          L2020A                 L2020A   
F                        PER-AVER               PER-AVER   
Units                    UMHOS/CM               UMHOS/CM   
1921-10-31              284.73438              357.93190   
1921-11-30              281.05875              354.92264   
1921-12-31              376.29092              432.88925   
1922-01-31              623.95900              635.65515   
1922-02-28              416.49713              465.80610   
..                            ...                    ...   
2021-05-31              393.38092              446.88086   
2021-06-30              383.89062              439.11115   
2021-07-31              446.86966              490.67210   
2021-08-31              464.96625              505.48776   
2021-09-30              438.54697              483.85828   

A                                                         \
B          BANKSEC_MAX14DAY_s0002 TRACYEC_MAX14DAY_s0002   
C                 SALINITY-APPROX        SALINITY-APPROX   
D                            1MON                   1MON   
E                          L2020A                 L2020A   
F                        PER-AVER               PER-AVER   
Units                    UMHOS/CM               UMHOS/CM   
1921-10-31              284.73438              357.93190   
1921-11-30              280.69608              354.62573   
1921-12-31              400.25623              452.50967   
1922-01-31              686.99084              687.25934   
1922-02-28              419.23160              468.04480   
..                            ...                    ...   
2021-05-31              395.85843              448.90918   
2021-06-30              383.63202              438.89940   
2021-07-31              459.99188              501.41525   
2021-08-31              473.36250              512.36176   
2021-09-30              424.56873              472.41430   

A                                                         \
B          BANKSEC_MAX14DAY_s0003 TRACYEC_MAX14DAY_s0003   
C                 SALINITY-APPROX        SALINITY-APPROX   
D                            1MON                   1MON   
E                          L2020A                 L2020A   
F                        PER-AVER               PER-AVER   
Units                    UMHOS/CM               UMHOS/CM   
1921-10-31              329.17500              394.31543   
1921-11-30              296.69278              367.72226   
1921-12-31              411.21050              461.47794   
1922-01-31              565.80280              588.04266   
1922-02-28              298.34177              369.07227   
..                            ...                    ...   
2021-05-31              366.52634              424.89500   
2021-06-30              373.53330              430.63160   
2021-07-31              482.44656              519.79890   
2021-08-31              493.06670              528.49360   
2021-09-30              457.05203              499.00840   

A                                                         \
B          BANKSEC_MAX14DAY_s0004 TRACYEC_MAX14DAY_s0004   
C                 SALINITY-APPROX        SALINITY-APPROX   
D                            1MON                   1MON   
E                          L2020A                 L2020A   
F                        PER-AVER               PER-AVER   
Units                    UMHOS/CM               UMHOS/CM   
1921-10-31              360.45148              419.92150   
1921-11-30              358.37662              418.22280   
1921-12-31              345.13815              407.38450   
1922-01-31              447.98657              491.58650   
1922-02-28              297.62354              368.48425   
..                            ...    

### X2

In [10]:
# Annual Average
april_x2_ann_avg = compute_annual_means(df, x2, units="KM", months=[4])
september_x2_ann_avg = compute_annual_means(df, x2, units="KM", months=[9])

# Annual CV
april_x2_ann_cv = compute_cv(df, x2, "April_X2_CV", months=[4], units="KM")
april_x2_ann_cv.index.name = 'Scenario'
september_x2_ann_cv = compute_cv(df, x2, "September_X2_CV", months=[9], units="KM")
september_x2_ann_cv.index.name = 'Scenario'

In [11]:
# Clean up dataframes for export
april_x2_ann_avg.columns = 'AprilX2_' + april_x2_ann_avg.columns.get_level_values('B').str.split('_').str[3]
april_x2_ann_avg.columns.name = None

september_x2_ann_avg.columns = 'SeptemberX2_' + september_x2_ann_avg.columns.get_level_values('B').str.split('_').str[3]
september_x2_ann_avg.columns.name = None

### Salinity at compliance points

In [12]:
salinity_compliance_points_indelta_df = create_subset_list(df, compliance_points_indelta)
salinity_compliance_points_export_df = create_subset_list(df, compliance_points_export)

In [13]:
# Clean up dataframes for export
salinity_compliance_points_indelta_df.columns = salinity_compliance_points_indelta_df.columns.get_level_values('B')
salinity_compliance_points_indelta_df.columns.name = None

salinity_compliance_points_export_df.columns = salinity_compliance_points_export_df.columns.get_level_values('B')
salinity_compliance_points_export_df.columns.name = None

### Tier calculation functions

In [14]:
def calc_indelta_tier(
    df,
    scenID,
    stations=["EM_EC_MONTH", "JP_EC_MONTH"],
    thresholds={"Top": 2500, "Mid": 1600, "Low": 900},
    tier_rules=OrderedDict([
        (1, {"LT_A": 0.75, "LT_B": None, "GT_C": 0.05}),
        (2, {"LT_A": 0.65, "LT_B": 0.75, "GT_C": 0.12}),
        (3, {"LT_A": 0.55, "LT_B": 0.65, "GT_C": 0.20}),
    ])
):
    """
    Calculate in-delta tier designation for a given scenario.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with salinity variables.
    scenID : str
        Scenario identifier.
    in_delta_vars : list of str, optional
        Variables to include (default: ["EM_EC_MONTH", "JP_EC_MONTH"]).
    thresholds : dict, optional
        Thresholds for salinity (default: {"Top": 2500, "Mid": 1600, "Low": 900}).
    tier_rules : dict, optional
        Rules for assigning tiers. Each tier is an ordered dict with keys "LT_A", "LT_B", "GT_C".
        Example (default):
        ([
            (1, {"LT_A": 0.75, "LT_B": None, "GT_C": 0.05}),
            (2, {"LT_A": 0.65, "LT_B": 0.75, "GT_C": 0.12}),
            (3, {"LT_A": 0.55, "LT_B": 0.65, "GT_C": 0.20}),
        ])        
    If no rule matches, returns tier = np.nan.
    """
    import pandas as pd
    idx = pd.IndexSlice

    tA, tB, tC = thresholds["Low"], thresholds["Mid"], thresholds["Top"]

    # get the data for this scenario
    selcols = [c for c in df.columns if scenID in c[1]]
    if len(selcols) < len(in_delta_vars):
        raise ValueError(f"Didn't find the salinity columns for scenario {scenID}")

    thisdat = df.loc[:, selcols]

    # store fractions for each variable
    fracs = {}
    for var in in_delta_vars:
        col = idx[:, f"{var}_{scenID}"]
        values = thisdat.loc[:, col].values

        fracs[var] = {
            "LT_A": sum(values < tA) / len(values),
            "LT_B": sum(values < tB) / len(values),
            "LT_C": sum(values < tC) / len(values),
            "GT_C": sum(values > tC) / len(values),
        }

    # aggregate across vars
    max_GT_C = max(v["GT_C"] for v in fracs.values())
    min_LT_A = min(v["LT_A"] for v in fracs.values())
    min_LT_B = min(v["LT_B"] for v in fracs.values())

    # apply tier rules in order
    for tier, rule in tier_rules.items():
        cond_A = min_LT_A >= rule["LT_A"] if rule["LT_A"] is not None else True
        cond_B = min_LT_B >= rule["LT_B"] if rule["LT_B"] is not None else True
        cond_C = max_GT_C < rule["GT_C"] if rule["GT_C"] is not None else True

        if cond_A and cond_B and cond_C:
            return tier

    # default if no rule matches
    return np.nan

In [15]:
def generate_salinity_tier_assignment_matrix(
    df,
    station_list=["EM", "JP"],
    thresholds={"Top": 2500, "Mid": 1600, "Low": 900},
    start_date="1921-10-01"
):
    def extract_scenario_id(colname):
        name = "_".join(colname) if isinstance(colname, tuple) else str(colname)
        match = re.search(r's\d{4}', name)
        return match.group(0) if match else None

    def extract_station_name(colname):
        name = "_".join(colname) if isinstance(colname, tuple) else str(colname)
        for st in station_list:
            if name.startswith(st + "_") or f"_{st}_" in name:
                return st
        return None

    def assign_tiers_by_scenario(df, date_series):
        tier_rows = []
        scenario_map = {}

        for col in df.columns:
            sid = extract_scenario_id(col)
            station = extract_station_name(col)
            if sid and station:
                scenario_map.setdefault(sid, {})[station] = col

        print(f"Found {len(scenario_map)} scenarios: {list(scenario_map.keys())}")

        for sid, col_dict in scenario_map.items():
            if not all(st in col_dict for st in station_list):
                print(f" Skipping {sid}: missing one or more station columns")
                continue

            df_scenario = pd.DataFrame(
                {st: df[col_dict[st]] for st in station_list},
                index=date_series
            )
            df_scenario["Year"] = df_scenario.index.year

            valid_rows = df_scenario.dropna(subset=station_list)
            if valid_rows.empty:
                print(f" Skipping {sid}: all data is NaN")
                continue

            yearly = valid_rows.groupby("Year")
            valid_years = list(yearly.groups.keys())
            total_years = len(valid_years)

            tier4_flag = False
            tier3_flag = False
            tier3_years_with_1month_over_mid = 0
            tier2_valid_years = 0
            tier1_valid_years = 0
            any_year_exceeds_mid = False

            for year, group in yearly:
                readings = {st: group[st] for st in station_list}

                if any((r > thresholds["Top"]).sum() >= 2 for r in readings.values()):
                    tier4_flag = True
                    break

                if any((r > thresholds["Mid"]).sum() >= 2 for r in readings.values()):
                    tier3_flag = True

                if any((r > thresholds["Mid"]).any() for r in readings.values()):
                    tier3_years_with_1month_over_mid += 1


                if any((r > thresholds["Mid"]).any() for r in readings.values()):
                    any_year_exceeds_mid = True
                else:
                    in_range_counts = [((r >= thresholds["Low"]) & (r <= thresholds["Mid"])).sum() for r in readings.values()]
                    if all(count >= 10 for count in in_range_counts):
                        tier2_valid_years += 1


                if all(((r < thresholds["Low"]).sum() == 12) for r in readings.values()):
                    tier1_valid_years += 1

            if total_years == 0:
                print(f" Scenario {sid}: No valid years with complete data.")
                continue

            if tier4_flag:
                tier = 4
            elif tier3_flag or (tier3_years_with_1month_over_mid / total_years > 0.05):
                tier = 3
            elif not any_year_exceeds_mid and (tier2_valid_years / total_years >= 0.95):
                tier = 2
            elif tier1_valid_years / total_years >= 0.95:
                tier = 1
            else:
                tier = None
                print(f" Scenario {sid} did not match any tier.")
                print(f"   Summary: tier3_flag={tier3_flag}, tier3_pct={tier3_years_with_1month_over_mid / total_years:.2f}, "
                      f"tier2_pct={tier2_valid_years / total_years:.2f}, tier1_pct={tier1_valid_years / total_years:.2f}, "
                      f"any_year_exceeds_mid={any_year_exceeds_mid}")
                continue

            print(f"→ Scenario {sid} assigned Tier {tier}")
            tier_rows.append({
                "Scenario": sid,
                "Salinity_Tier": tier
            })

        return pd.DataFrame(tier_rows, columns=["Scenario", "Salinity_Tier"])

    df = df.copy()
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        df.index = pd.date_range(start=start_date, periods=len(df), freq="MS")

    date_series = df.index
    tier_df = assign_tiers_by_scenario(df, date_series)

    if tier_df.empty:
        print(" No valid scenario-station pairs were found.")
        return pd.DataFrame(columns=["Salinity_Tier"])

    return tier_df.set_index("Scenario")


### In-Delta tier assignment (new version):

In [16]:
tiers = {} #<- dictionary to store results
for scenID in index_names: # iterate through the list of scenario IDs
    # call the function defined above
    tiers[scenID] = calc_indelta_tier(df = in_delta_df, scenID = scenID, stations = in_delta_vars, thresholds =  indelta_thresholds, tier_rules= indelta_rules)
    # print out the progress
    print(f"assigned tier {tiers[scenID]} to scenario {scenID}")

# create a dataframe from the dictionary - index is the scenario id, 
# single column is the tier value
tier_indelta_df = pd.DataFrame.from_dict(tiers, orient='index', columns=['Salinity_Tier']) #, index='ScenarioID')
tier_indelta_df.index.name = 'ScenarioID' #rename the column

assigned tier 2 to scenario s0001
assigned tier 2 to scenario s0002
assigned tier 3 to scenario s0003
assigned tier 3 to scenario s0004
assigned tier 3 to scenario s0005
assigned tier 2 to scenario s0006
assigned tier 2 to scenario s0007
assigned tier 2 to scenario s0008
assigned tier 2 to scenario s0009
assigned tier 2 to scenario s0010
assigned tier 2 to scenario s0011
assigned tier 2 to scenario s0012
assigned tier 3 to scenario s0013
assigned tier 3 to scenario s0014
assigned tier 2 to scenario s0015
assigned tier 3 to scenario s0016
assigned tier 1 to scenario s0018
assigned tier 2 to scenario s0019
assigned tier 2 to scenario s0020
assigned tier 2 to scenario s0021
assigned tier 2 to scenario s0022
assigned tier 2 to scenario s0023
assigned tier 3 to scenario s0024
assigned tier 2 to scenario s0025
assigned tier 2 to scenario s0027
assigned tier 1 to scenario s0029
assigned tier 1 to scenario s0030
assigned tier 1 to scenario s0039
assigned tier 1 to scenario s0042
assigned tier 

In [17]:
tier_indelta_df

,Salinity_Tier
ScenarioID,
s0001,2
s0002,2
s0003,3
s0004,3
s0005,3
s0006,2
s0007,2
s0008,2
s0009,2


### In-Delta tier assignment (old version):

In [18]:
# tier_indeltaold_df = generate_salinity_tier_assignment_matrix(
#     df=in_delta_df,
#     station_list=indelta_station_list,
#     thresholds=indelta_thresholds,
#     start_date="1921-10-01"
# )

In [19]:
# tier_indeltaold_df

### Export tier assignment:

In [20]:
tier_export_df = generate_salinity_tier_assignment_matrix(
    df=export_df,
    station_list=export_station_list,
    thresholds=export_thresholds,
    start_date="1921-10-01"
)

Found 35 scenarios: ['s0001', 's0002', 's0003', 's0004', 's0005', 's0006', 's0007', 's0008', 's0009', 's0010', 's0011', 's0012', 's0013', 's0014', 's0015', 's0016', 's0018', 's0019', 's0020', 's0021', 's0022', 's0023', 's0024', 's0025', 's0027', 's0029', 's0030', 's0039', 's0042', 's0046', 's0047', 's0051', 's0056', 's0062', 's0063']
→ Scenario s0001 assigned Tier 1
→ Scenario s0002 assigned Tier 1
→ Scenario s0003 assigned Tier 1
→ Scenario s0004 assigned Tier 1
→ Scenario s0005 assigned Tier 1
→ Scenario s0006 assigned Tier 1
→ Scenario s0007 assigned Tier 1
→ Scenario s0008 assigned Tier 1
→ Scenario s0009 assigned Tier 1
→ Scenario s0010 assigned Tier 1
→ Scenario s0011 assigned Tier 1
→ Scenario s0012 assigned Tier 1
→ Scenario s0013 assigned Tier 1
→ Scenario s0014 assigned Tier 1
→ Scenario s0015 assigned Tier 1
→ Scenario s0016 assigned Tier 1
→ Scenario s0018 assigned Tier 1
→ Scenario s0019 assigned Tier 1
→ Scenario s0020 assigned Tier 1
→ Scenario s0021 assigned Tier 1
→ Sc

In [21]:
tier_export_df

,Salinity_Tier
Scenario,
s0001,1
s0002,1
s0003,1
s0004,1
s0005,1
s0006,1
s0007,1
s0008,1
s0009,1


## Save files

In [22]:
april_x2_ann_avg.to_csv(april_x2_ann_avg_path, index=True)
april_x2_ann_cv.to_csv(april_x2_ann_cv_path, index=True)
september_x2_ann_avg.to_csv(september_x2_ann_avg_path, index=True)
september_x2_ann_cv.to_csv(september_x2_ann_cv_path, index=True)
salinity_compliance_points_indelta_df.to_csv(salinity_compliance_points_indelta_path, index=True)
salinity_compliance_points_export_df.to_csv(salinity_compliance_points_export_path, index=True)
tier_indelta_df.to_csv(indelta_salinity_output_path, index=True)
tier_export_df.to_csv(export_salinity_output_path, index=True)

In [23]:
print(salinity_tiers_output_dir)
print(salinity_data_output_dir)

C:\Dino\COEQWAL\COEQWAL_GIT\CalSim3_Model_Runs\Scenarios\Performance_Metrics\Tiered_Outcome_Measures\Salinity\Tiers
C:\Dino\COEQWAL\COEQWAL_GIT\CalSim3_Model_Runs\Scenarios\Performance_Metrics\Metrics\Salinity


In [24]:
print("Done!")

Done!
